<a href="https://colab.research.google.com/github/BhoomiMahna/outfit-vibe-rater/blob/main/pattern_density_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded=files.upload()


Saving outfits_final.zip to outfits_final.zip


In [ ]:
import os
import zipfile

zip_path='outfits_final.zip'
extract_path='outfits_final'

if not os.path.exists(extract_path):
  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    print(f"Files extracted to '{extract_path}' folder.")
else:
    print("Files already extracted.")

Files extracted to 'outfits_final' folder.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers,models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

data_dir ='outfits_final/outfits/pattern density'
img_size=(260,260)
batch_size=32
model_save_path ="pattern_density_final.h5"

train_ds=tf.keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode ='categorical',
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
num_classes=len(class_names)
print(f"found {num_classes} classes: {class_names}")

Found 1277 files belonging to 3 classes.
Using 1022 files for training.
Found 1277 files belonging to 3 classes.
Using 255 files for validation.
found 3 classes: ['checked', 'printed', 'solid']


In [ ]:
data_augmentation = models.Sequential([
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
], name ="data_augmentation")

AUTOTUNE =tf.data.AUTOTUNE
train_ds=train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds=val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

data_dir = 'outfits_final/outfits/pattern density'
img_size = (260, 260) # Optimal size for EfficientNetV2B2
batch_size = 32
model_save_path = "pattern_density_best_model.h5"

print("Loading and preparing datasets...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode='categorical',
    validation_split=0.2,
    subset='training',
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Found {num_classes} classes: {class_names}")


# --- 3. Augmentation and Performance Tuning ---
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
], name="data_augmentation")

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
print("✅ Data augmentation and performance tuning are set up.")


# --- 4. Build the Model (EfficientNetV2B2) ---
base_model = tf.keras.applications.EfficientNetV2B2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

inputs = layers.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = tf.keras.applications.efficientnet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)
print("✅ Model built with EfficientNetV2B2 base.")


# --- 5. Initial Training ---
initial_epochs = 10
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- Starting Initial Training (Training the head) ---")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=initial_epochs
)


# --- 6. Fine-Tuning Setup ---
base_model.trainable = True
for layer in base_model.layers[:-50]: # Freeze all but the top 50 layers
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # Very low learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks to save the best model and stop early
checkpoint_cb = ModelCheckpoint(
    model_save_path,
    save_best_only=True,
    monitor="val_accuracy",
    mode="max"
)
early_stopping_cb = EarlyStopping(
    patience=3,
    monitor="val_loss",
    restore_best_weights=True
)
print("✅ Model re-compiled for fine-tuning.")


# --- 7. Continue Training with Fine-Tuning ---
print("\n--- Starting Fine-Tuning ---")
fine_tune_epochs = 15
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    callbacks=[checkpoint_cb, early_stopping_cb]
)


# --- 8. Final Evaluation ---
print("\n--- Evaluating Final Model ---")
# Load the best model saved by ModelCheckpoint for the most reliable result
best_model = tf.keras.models.load_model(model_save_path)
final_loss, final_accuracy = best_model.evaluate(val_ds)
print(f"\n✅ Final Validation Accuracy for Pattern Density Model: {final_accuracy:.4f}")

Loading and preparing datasets...
Found 1277 files belonging to 3 classes.
Using 1022 files for training.
Found 1277 files belonging to 3 classes.
Using 255 files for validation.
Found 3 classes: ['checked', 'printed', 'solid']
✅ Data augmentation and performance tuning are set up.
✅ Model built with EfficientNetV2B2 base.

--- Starting Initial Training (Training the head) ---
Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 390ms/step - accuracy: 0.4134 - loss: 1.1145 - val_accuracy: 0.7059 - val_loss: 0.7572
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 138ms/step - accuracy: 0.6851 - loss: 0.7845 - val_accuracy: 0.8157 - val_loss: 0.6136
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 142ms/step - accuracy: 0.7504 - loss: 0.6816 - val_accuracy: 0.8196 - val_loss: 0.5360
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 140ms/step - accuracy: 0.7733 - loss: 0.5943 - val_accuracy: 0.8314 - val_loss: 0.4979
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 140ms/step - accuracy: 0.7958 - loss: 0.5498 - val_accuracy: 0.843

32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 352ms/step - accuracy: 0.8386 - loss: 0.4996 - val_accuracy: 0.8431 - val_loss: 0.4306
Epoch 11/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.8525 - loss: 0.4910

32/32 ━━━━━━━━━━━━━━━━━━━━ 15s 193ms/step - accuracy: 0.8527 - loss: 0.4903 - val_accuracy: 0.8667 - val_loss: 0.4172
Epoch 12/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.8512 - loss: 0.4716

32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 197ms/step - accuracy: 0.8521 - loss: 0.4707 - val_accuracy: 0.8784 - val_loss: 0.3973
Epoch 13/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.8832 - loss: 0.4404

32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 198ms/step - accuracy: 0.8832 - loss: 0.4400 - val_accuracy: 0.8863 - val_loss: 0.3812
Epoch 14/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 194ms/step - accuracy: 0.8716 - loss: 0.4298 - val_accuracy: 0.8863 - val_loss: 0.3688
Epoch 15/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.8899 - loss: 0.3982

32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 196ms/step - accuracy: 0.8903 - loss: 0.3975 - val_accuracy: 0.8902 - val_loss: 0.3550
Epoch 16/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 180ms/step - accuracy: 0.8799 - loss: 0.3946 - val_accuracy: 0.8902 - val_loss: 0.3414
Epoch 17/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 178ms/step - accuracy: 0.8929 - loss: 0.3860 - val_accuracy: 0.8863 - val_loss: 0.3311
Epoch 18/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 181ms/step - accuracy: 0.8951 - loss: 0.3593 - val_accuracy: 0.8824 - val_loss: 0.3239
Epoch 19/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 178ms/step - accuracy: 0.8949 - loss: 0.3434 - val_accuracy: 0.8902 - val_loss: 0.3164
Epoch 20/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 178ms/step - accuracy: 0.8897 - loss: 0.3492 - val_accuracy: 0.8902 - val_loss: 0.3090
Epoch 21/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 176ms/step - accuracy: 0.9033 - loss: 0.3238 - val_accuracy: 0.8902 - val_loss: 0.3047
Epoch 22/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 179ms/step - accuracy: 0.8998 - loss: 0.3391 - val_accuracy: 0.890

32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 195ms/step - accuracy: 0.9106 - loss: 0.3217 - val_accuracy: 0.8941 - val_loss: 0.2875
Epoch 25/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.9109 - loss: 0.2939

32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 194ms/step - accuracy: 0.9113 - loss: 0.2933 - val_accuracy: 0.9020 - val_loss: 0.2848

--- Evaluating Final Model ---


8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 107ms/step - accuracy: 0.8969 - loss: 0.2981

✅ Final Validation Accuracy for Pattern Density Model: 0.9020
